In [1]:
import json
import yaml
import boto3
import logging
from pathlib import Path

In [ ]:
# Setup logging
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# global constants
CONFIG_FILE: str = "config.yml"

In [ ]:
# fetch the current AWS region
region = boto3.client('sts').meta.region_name # change to another region if you do not want
# the region to be dynamically fetched
logger.info(f"Current AWS region: {region}")
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = region)

In [ ]:
config = yaml.safe_load(Path(CONFIG_FILE).read_text())
logger.info(f"config=\n{json.dumps(config, indent=2)}")

In [ ]:
import re
from typing import List

def extract_mustache_keys(text: str) -> List[str]:
    """
    Extract all mustache-style keys ({{key}}) from the input text.
    
    Args:
        text (str): Input text containing mustache syntax
        
    Returns:
        List[str]: List of extracted keys without the curly braces
    """
    # The pattern looks for:
    # \{\{ - literal {{ (escaped because { is special in regex)
    # (.*?) - any characters (non-greedy match)
    # \}\} - literal }}
    pattern = r'\{\{(.*?)\}\}'
    
    # Find all matches and extract the group inside the braces
    matches = re.findall(pattern, text)
    
    # Strip whitespace from each match
    return [match.strip() for match in matches]

In [ ]:
def create_bedrock_prompt(name: str, description: str, text: str, parameters: dict, model_id: str):
    """
    Creates a prompt configuration for Amazon Bedrock using specified parameters.
    
    This function constructs a prompt template with inference settings and template configuration
    for use with Amazon Bedrock's language models. It automatically extracts input variables 
    from mustache-style placeholders in the template text.

    Args:
        name (str): The name identifier for the prompt template
        description (str): A description of the prompt template's purpose
        text (str): The template text containing mustache-style variables (e.g., {{variable}})
        parameters (dict): Configuration parameters including:
            - max_tokens (int): Maximum number of tokens in the response
            - temperature (float): Sampling temperature for text generation
        model_id (str): The identifier of the Bedrock model to use

    Returns:
        dict: The response from the Bedrock create_prompt API call, containing the created
              prompt configuration details

    Examples:
        >>> parameters = {
        ...     'max_tokens': 100,
        ...     'temperature': 0.7
        ... }
        >>> create_bedrock_prompt(
        ...     name="test_prompt",
        ...     description="A test prompt",
        ...     text="Hello {{name}}, how are you?",
        ...     parameters=parameters,
        ...     model_id="anthropic.claude-v2"
        ... )

    Notes:
        - The function automatically creates a default variant named "default_variant"
        - Input variables are automatically extracted from {{mustache}} syntax in the template text
        - Logs the API response using the logger module
        - Requires the bedrock_agent and logger to be properly configured
    """
    
    default_variant_name = "default_variant"
    input_variables = [dict(name=k) for k in extract_mustache_keys(text)]
    variant = {
            "inferenceConfiguration": {
            "text": {
                "maxTokens": parameters['max_tokens'],
                "temperature": parameters['temperature'],
                }
            },
            "modelId": model_id,
            "name": default_variant_name,
            "templateConfiguration": {
                "text": {
                    "inputVariables": input_variables,
                    "text": text
                }
            },
            "templateType": "TEXT"
        }
    response = bedrock_agent.create_prompt(name=name,
                                           description=description,
                                           variants=[variant],
                                           defaultVariant=default_variant_name)
    logger.info(f"response={json.dumps(response, indent=2)}")
    return response


    


In [ ]:
for model_family, info in config['prompt_templates'].items():
    for model_id in info['models']:
        name = config['prompt_info']['name'].format(model_id=model_id)
        name = re.sub('[:\.]', '-', name)
        try:
            response = create_bedrock_prompt(name,
                                             config['prompt_info']['description'],
                                             info['text'],
                                             config['inference_parameters'],
                                             model_id)
        except Exception as e:
            logger.error(f"exception occurred while creating prompt, name={name}, exception={e}")